<a href="https://www.kaggle.com/code/somewhatjustin/titanic?scriptVersionId=164018831" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# TODO: Add Cross-Validation for depth

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Fill NaNs without using inplace=True
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())

# Fit and transform the data for training and testing datasets
train_data['Age_normalized'] = scaler.fit_transform(train_data[['Age']].values.reshape(-1, 1))
test_data['Age_normalized'] = scaler.transform(test_data[['Age']].values.reshape(-1, 1))

# Note: Use scaler.fit_transform() on the training data to fit the scaler and transform the data.
# Use scaler.transform() on the test data to apply the same scaling based on the training data.

In [ ]:
# Initialize Fare Scaler
fareScaler = MinMaxScaler(feature_range=(0,1))

# Fill NaNs without using inplace=True
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].mean())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())

# Fit and transform the data for training and testing
train_data['Fare'] = fareScaler.fit_transform(train_data[['Fare']].values.reshape(-1,1))
test_data['Fare'] = fareScaler.transform(test_data[['Fare']].values.reshape(-1,1))

In [ ]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Age_normalized", "Fare"]
#features = ["Pclass", "Sex", "SibSp", "Parch", "Age_normalized", "Fare"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, max_depth=3, random_state=0).fit(X, y)
submissionPredictions = model.predict(X_test)

In [ ]:
# CHECK FOR OVERFITTING
from sklearn.metrics import accuracy_score

training_predictions = model.predict(X)
training_truth = y.to_numpy()

accuracy = accuracy_score(training_truth, training_predictions)
print(f"Accuracy:{accuracy}")

In [ ]:
# CROSS-VALIDATION

def splitIntoGroups(data, numGroups):
    df_shuffled = data.sample(frac=1, random_state=21).reset_index(drop=True)
    groups = np.array_split(df_shuffled, 5)
    return groups

def singleHyperTrain(train, test):
    topAccuracy = {"accuracy": 0}
    optimalWeights = []
    estimators = [1, 10, 100, 1000, 10000]
    max_depth = [1, 2, 3, 4, 5]
    for i in estimators:
        for j in max_depth:
            y = train["Survived"]
            y_test = test["Survived"]
            # features = ["Pclass", "Sex", "SibSp", "Parch", "Age_normalized", "Embarked", "Fare", "hasAge"]
            features = ["Pclass", "Sex", "SibSp", "Parch", "Age_normalized", "Fare"]
            X = pd.get_dummies(train[features])
            X_test = pd.get_dummies(test[features])
            testModel = GradientBoostingClassifier(n_estimators=i, learning_rate=1.0, max_depth=j, random_state=0).fit(X, y)
            predictions = testModel.predict(X_test)
            accuracy = accuracy_score(y_test.to_numpy(), predictions)
            if accuracy > topAccuracy["accuracy"]:
                topAccuracy = {"accuracy": accuracy, "estimators": i, "max_depth": j}
            print(f"{accuracy} with est: {i}, max depth: {j}")
    print(f"top accuracy: {topAccuracy['accuracy']} with estimators = {topAccuracy['estimators']} and {topAccuracy['max_depth']} max depth")
        
def crossTrain(data):
    for i in range(0, len(data)):
        trainGroup = pd.concat([groupsOfData[j] for j in range(len(groupsOfData)) if j != i])
        singleHyperTrain(trainGroup, groupsOfData[i])
    
        
groupsOfData = splitIntoGroups(train_data, 5)
crossTrain(groupsOfData)

In [ ]:
# CREATE PREDICTIONS
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': submissionPredictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")